In [156]:
import os
import numpy as np
import pandas as pd
import pickle

import bqplot as bq
from ipywidgets import Layout, Box

In [142]:
from bqplot import LinearScale, Axis, Lines, Scatter, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML

In [5]:
ddir = '/epyc/projects/sso-lc/notebooks/aug_29_2019'
filename = 't.csv'
d = pd.read_csv(os.path.join(ddir, filename))
d[0:3]

,ztfname_x,Nobs,N_rejected,Nobs_g,Nobs_r,Nobs_i,Nights,g-r,r-i,fit_period,...,binary,priv,pole,sparse,widefield,objId,Frequency,err,check,pmod
0,7641,415,0,94,321,0,127,0.592583,-999.000000,27.804073,...,NaN,NaN,NaN,0.0,0.0,7641.0,0.036010,0.001227,0,0.158503
1,65245,335,0,12,312,11,42,0.525240,0.272383,9.806869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.408620
2,65257,291,0,23,258,10,55,0.485952,0.172055,17.856068,...,NaN,NaN,NaN,0.0,1.0,65257.0,0.056085,0.001462,0,0.744003


In [140]:
both = d.query('period > 0')

In [150]:
x_sc = LinearScale(min=0, max=60)
y_sc = LinearScale(min=0, max=60)

x_data = both.fit_period.values
y_data = both.period.values

scatter_chart = Scatter(x=x_data, y=y_data, scales= {'x': x_sc, 'y': y_sc}, colors=['dodgerblue'],
                       interactions={'click': 'select'},
                        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                       unselected_style={'opacity': 0.5})

ax_x = Axis(scale=x_sc, tick_format='0.0f', label='Fit period')
ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='0.0f', label='ALCDEF period')

Figure(marks=[scatter_chart], axes=[ax_x, ax_y])

Figure(axes=[Axis(label='Fit period', scale=LinearScale(max=60.0, min=0.0), tick_format='0.0f'), Axis(label='A…

In [155]:
cols = ['ztfname_x', 'Nobs', 'Nights', 'mag_med', 'sig_med', 'fit_period', 'period', 'fit_amp', 'AmpMax', 'U']
both.iloc[scatter_chart.selected][cols]

,ztfname_x,Nobs,Nights,mag_med,sig_med,fit_period,period,fit_amp,AmpMax,U
294,5023,34,22,17.61125,0.054305,1.291727,5.402,0.244116,0.12,2+
167,1871,50,36,18.92125,0.113877,1.131771,6.520,0.206518,0.13,2-
332,11252,31,24,17.82790,0.062938,1.455077,9.150,0.197925,0.18,2
310,1143,33,24,15.94960,0.033165,1.024682,10.114,0.231988,0.22,3
290,18493,35,22,18.57630,0.094450,1.285766,14.430,0.179733,0.18,2
259,6002,37,25,18.48200,0.093232,1.066499,12.918,0.226818,0.18,3-
255,18137,38,23,18.50130,0.082527,1.045026,16.200,0.296236,0.30,3
334,63287,30,24,19.68870,0.136043,1.637695,17.682,0.461933,0.10,NaN
166,5638,50,31,18.03955,0.075208,1.577400,19.400,0.103816,0.14,2
29,32496,116,64,18.18890,0.079227,1.037457,23.340,0.074911,0.19,3-


In [174]:
def plot_scatter(x=[], y=[], zid=[], color='red', filt=''):
    '''Create and return Scatter plot'''
    #TODO: tooltip format with all data  
    tooltip = bq.Tooltip(fields=['x', 'y', 'zid'], formats=['.2f', '.2f', ''],
                        labels=['fit_period', 'lcd_period', 'ztfname'])
    sc_x = bq.LinearScale(min=0, max=50)
    sc_y = bq.LinearScale(min=0, max=50)
    
    scatt = bq.Scatter(
            scales={'x': sc_x, 'y': sc_y},
            tooltip=tooltip,
            tooltip_style={'opacity': 0.5},
            interactions={'hover': 'tooltip'},
            unhovered_style={'opacity': 0.5},
            selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
            unselected_style={'opacity': 0.5},
            display_legend=False)
    scatt.colors = [color]
    scatt.label = filt
    if ((y != [])):
        scatt.x = x
        scatt.y = y
    #scatt.on_element_click(display_info)

    return scatt

In [175]:
both['fit_period'].values[0:3], both['period'].values[0:3], both['ztfname_x'].values[0:3]

(array([27.804073, 17.856068, 10.214489]),
 array([27.77 , 17.83 , 10.212]),
 array(['7641', '65257', '2920'], dtype=object))

In [176]:
scat = plot_scatter(both['fit_period'].values, both['period'].values, 
                    both['ztfname_x'].values, color='dodgerblue')

/astro/users/ajc26/.conda/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [177]:
sc_x = bq.LinearScale()
sc_y = bq.LinearScale()

xax = bq.Axis(label='Fit Period', scale=sc_x,
                grid_lines='solid',
                label_location="middle")
xax.tick_style={'stroke': 'black', 'font-size': 12}

yax = bq.Axis(label='ACLDEF Period', scale=sc_y,
                orientation='vertical', tick_format='0.1f',
                grid_lines='solid', label_location="middle")
yax.tick_style={'stroke': 'black', 'font-size': 12}

ax_x = Axis(scale=x_sc, tick_format='0.0f', label='Fit period')
ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='0.0f', label='ALCDEF period')

ax_x = Axis(scale=sc_x, tick_format='0.0f', label='Fit period')
ax_y = Axis(scale=sc_y, orientation='vertical', tick_format='0.0f', label='ALCDEF period')


#panzoom = bq.PanZoom(scales={'x': [sc_x], 'y': [sc_y]})
bq.Figure(#axes=[ax_x, ax_y], 
          marks=[scat],
            layout=Layout(width='500px', height='500px'),
            fig_margin = {'top': 0, 'bottom': 40, 'left': 50, 'right': 0},
             #legend_location='top-right',
             )

Figure(fig_margin={'top': 0, 'bottom': 40, 'left': 50, 'right': 0}, layout=Layout(height='500px', width='500px…

In [48]:
help(sc_x)

Help on LinearScale in module bqplot.scales object:

class LinearScale(Scale)
 |  A linear scale.
 |  
 |  An affine mapping from a numerical domain to a numerical range.
 |  
 |  Attributes
 |  ----------
 |  min: float or None (default: None)
 |      if not None, min is the minimal value of the domain
 |  max: float or None (default: None)
 |      if not None, max is the maximal value of the domain
 |  rtype: string (class-level attribute)
 |      This attribute should not be modifed. The range type of a linear
 |      scale is numerical.
 |  dtype: type (class-level attribute)
 |      the associated data type / domain type
 |  precedence: int (class-level attribute, default_value=2)
 |      attribute used to determine which scale takes precedence in cases when
 |      two or more scales have the same rtype and dtype.
 |      default_value is 2 because for the same range and domain types,
 |      LinearScale should take precedence.
 |  stabilized: bool (default: False)
 |      if set